In [2]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight

Using TensorFlow backend.


In [3]:
gc.enable()

In [4]:
steps = 5
height = 108
width = 108

In [5]:
#change the path to where you save the input numpy arrays
lstm = np.array(glob.glob('E:/notes_database/lstm_inputs_2/*'))
cnn = np.array(glob.glob('E:/notes_database/cnn_inputs_2/*'))
y_train = np.array(glob.glob('E:/notes_database/y_train_2/*'))

In [6]:
#The function analyzes the actual accuracy
def load_test_score(model, cnn_test, lstm_test, y_test):
    prediction = model.predict([cnn_test, lstm_test])
    error = (prediction - y_test) ** 2
    error = (error/y_test.shape[0]).sum()
    wrong = 0
    for x in range(prediction.shape[0]):
        for y in range(prediction.shape[1]):
            if abs(prediction[x][y] - y_test[x][y]) > 0.1:
                wrong += 1
                break
    accuracy = 1 - wrong/(prediction.shape[0])
    return error, accuracy

In [7]:
def inputs_to_model(order, num, start, ending):
    cnn_inputs = np.load(cnn[order[num][0]])
    labels_inputs = np.load(y_train[order[num][0]])
    lstm_inputs = np.load(lstm[order[num][0]])
    
    count = 0
    ce = 0
    for file in order[num][start+1:ending]:
        if count == ce + 50:
            print(count)
            ce = count
        count+=1
        
        lstm_inputs = np.append(lstm_inputs, np.load(lstm[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(y_train[file]), axis=0)
        
    return lstm_inputs, cnn_inputs, labels_inputs

In [8]:
def main(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [9]:
def main_2(optimizer, loss):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = Dropout(0.3)(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = Flatten()(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32, return_sequences=True)(lstm_layers)
    lstm_layers = LSTM(64, return_sequences=True)(lstm_layers)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

In [ ]:
def class_weights(y):
    samples = []  
    for i in range(y.shape[0]):
        pos = np.where(y[i] == 1)[0][0]
        samples.append(pos)
    samples = np.array(samples)
    classes = np.unique(samples)
    class_weights = class_weight.compute_class_weight('balanced', classes, samples)
    return classes, class_weights

In [10]:
def train_test_whole(classifier, num, name, epochs=100, batchsize=32):
    lstm_test, cnn_test, labels_test = inputs_to_model(test_order, num, 0, test_order[num].shape[0]+1)
    lstm_inputs, cnn_inputs, labels_inputs = inputs_to_model(train_order, num, 0, train_order[num].shape[0]+1)
    
    sample_weights = class_weight.compute_sample_weight('balanced', labels_inputs)
    classifier.fit([cnn_inputs, lstm_inputs], labels_inputs, epochs=epochs, batch_size=batchsize, verbose=1,
                  validation_data = ([cnn_test, lstm_test], labels_test), sample_weight=sample_weights)
    
    #CV_loss, accuracy = load_test_score(classifier, cnn_test, lstm_test, y_test)
    #print('val_loss: ', CV_loss)
    #print('val_accuarcy', accuracy)
    
    classifier.save_weights(name, overwrite=True)   
    del classifier
    K.clear_session()

In [11]:
#Generate the CV sets
kf = KFold(n_splits=3, random_state=7)

train_order = []
test_order = []
length = np.arange(len(lstm))

for x,y in kf.split(length[100:]):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [11]:
adam = keras.optimizers.Adam(lr=0.01)
classifier = main(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-spectrums2-temp-lr0.01.hdf5', epochs=50)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


50
Train on 35791 samples, validate on 18593 samples
Epoch 1/50
35791/35791 [==============================] - 40s 1ms/step - loss: 0.1284 - acc: 0.9549 - val_loss: 0.1061 - val_acc: 0.9775
Epoch 2/50
35791/35791 [==============================] - 37s 1ms/step - loss: 0.0600 - acc: 0.9813 - val_loss: 0.0508 - val_acc: 0.9854
Epoch 3/50
35791/35791 [==============================] - 37s 1ms/step - loss: 0.0503 - acc: 0.9845 - val_loss: 0.0705 - val_acc: 0.9774
Epoch 4/50
35791/35791 [==============================] - 37s 1ms/step - loss: 0.0460 - acc: 0.9860 - val_loss: 0.0483 - val_acc: 0.9853
Epoch 5/50
35791/35791 [==============================] - 38s 1ms/step - loss: 0.0429 - acc: 0.9871 - val_loss: 0.0548 - val_acc: 0.9847
Epoch 6/50
35791/35791 [==============================] - 37s 1ms/step - loss: 0.0402 - acc: 0.9879 - val_loss: 0.0347 - val_acc: 0.9901
Epoch 7/50
35791/35791 [==============================] - 37s 1ms/step - loss: 0.0389 - acc: 0.9884 - val_loss: 0.0468 - val_

32524

In [11]:
def train_test_split(num, name, epochs=100, batchsize=32, optimizer='adam', loss='mean_squared_error',
                     metrix='accuracy', split=2):
    classifier = main(optimizer, loss, metrix)
    
    lstm_test, cnn_test, labels_test = inputs_to_model(test_order, num, 0, test_order[num].shape[0]+1)
    for epoch in range(epochs):
        print("""
        
        THIS IS EPOCH {}""".format(epoch))
        for i in range(split):
            lstm,cnn,labels = inputs_to_model(train_order, num, (train_order[num].shape[0]*i)//split, 
                                              (train_order[num].shape[0]*(i+1))//split+1)
            sample_weights = class_weight.compute_sample_weight('balanced', labels)
            classifier.fit([cnn, lstm], labels, epochs=1, batch_size=batchsize, verbose=1,
                  validation_data = ([cnn_test, lstm_test], labels_test), sample_weight=sample_weights)
            
            # To save memory, it's possible to train the model first then test on the validation data.
            #CV_loss, accuracy = load_test_score(classifier, cnn_test, lstm_test, labels_test)
            #print('val_loss: ', CV_loss)
            #print('val_accuarcy', accuracy)
    
    classifier.save_weights(name, overwrite=True)
    del classifier
    K.clear_session()

In [12]:
#Generate the CV sets
kf = KFold(n_splits=3, random_state=7)

train_order = []
test_order = []
length = np.arange(len(lstm))

for x,y in kf.split(length):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [13]:
adam = keras.optimizers.Adam(lr=0.01)
classifier = main(optimizer=adam, loss='binary_crossentropy')
train_test_split(classifier, 0, 'weight-split-test.hdf5', epochs=10)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


50
50
Train on 36274 samples, validate on 36454 samples
Epoch 1/1
36274/36274 [==============================] - 46s 1ms/step - loss: 0.1278 - acc: 0.9552 - val_loss: 0.0896 - val_acc: 0.9734
50
Train on 36319 samples, validate on 36454 samples
Epoch 1/1
36319/36319 [==============================] - 43s 1ms/step - loss: 0.0632 - acc: 0.9801 - val_loss: 0.1186 - val_acc: 0.9678
50
Train on 36274 samples, validate on 36454 samples
Epoch 1/1
36274/36274 [==============================] - 43s 1ms/step - loss: 0.0529 - acc: 0.9838 - val_loss: 0.0576 - val_acc: 0.9822
50
Train on 36319 samples, validate on 36454 samples
Epoch 1/1
36319/36319 [==============================] - 43s 1ms/step - loss: 0.0480 - acc: 0.9855 - val_loss: 0.0372 - val_acc: 0.9889
50
Train on 36274 samples, validate on 36454 samples
Epoch 1/1
36274/36274 [==============================] - 43s 1ms/step - loss: 0.0450 - acc: 0.9866 - val_loss: 0.0399 - val_acc: 0.9883
50
Train on 36319 samples, validate on 36454 samples

32524

In [15]:
adam = keras.optimizers.Adam(lr=0.01)
classifier = main(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-spectrums2-all-lr0.01.hdf5', epochs=30)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


50
100
50
Train on 72054 samples, validate on 36454 samples
Epoch 1/30
72054/72054 [==============================] - 77s 1ms/step - loss: 0.0953 - acc: 0.9676 - val_loss: 0.0518 - val_acc: 0.9847
Epoch 2/30
72054/72054 [==============================] - 75s 1ms/step - loss: 0.0493 - acc: 0.9849 - val_loss: 0.0631 - val_acc: 0.9825
Epoch 3/30
72054/72054 [==============================] - 74s 1ms/step - loss: 0.0427 - acc: 0.9872 - val_loss: 0.1129 - val_acc: 0.9727
Epoch 4/30
72054/72054 [==============================] - 75s 1ms/step - loss: 0.0387 - acc: 0.9885 - val_loss: 0.0406 - val_acc: 0.9872
Epoch 5/30
72054/72054 [==============================] - 75s 1ms/step - loss: 0.0360 - acc: 0.9893 - val_loss: 0.0618 - val_acc: 0.9876
Epoch 6/30
72054/72054 [==============================] - 74s 1ms/step - loss: 0.0339 - acc: 0.9901 - val_loss: 0.0323 - val_acc: 0.9912
Epoch 7/30
72054/72054 [==============================] - 75s 1ms/step - loss: 0.0324 - acc: 0.9905 - val_loss: 0.0277

52726

In [14]:
adam = keras.optimizers.Adam(lr=0.01)
classifier = main_2(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-spectrums2-main_2-lr0.01.hdf5', epochs=30)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  import sys
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  # This is added back by InteractiveShellApp.init_path()
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


50
50
100
Train on 72054 samples, validate on 36454 samples
Epoch 1/30
72054/72054 [==============================] - 120s 2ms/step - loss: 0.0976 - acc: 0.9665 - val_loss: 0.0847 - val_acc: 0.9715
Epoch 2/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0510 - acc: 0.9844 - val_loss: 0.1273 - val_acc: 0.9759
Epoch 3/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0431 - acc: 0.9871 - val_loss: 0.1006 - val_acc: 0.9826
Epoch 4/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0384 - acc: 0.9886 - val_loss: 0.0864 - val_acc: 0.9866
Epoch 5/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0353 - acc: 0.9896 - val_loss: 0.1546 - val_acc: 0.9788
Epoch 6/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0332 - acc: 0.9903 - val_loss: 0.1435 - val_acc: 0.9763
Epoch 7/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0315 - acc: 0.9908 - val_loss:

64158